# OilyGiant

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
Crear un modelo para predecir el volumen de reservas en pozos nuevos.
Elegir los pozos petrolíferos que tienen los valores estimados más altos.
Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

Condiciones:

- Solo se debe usar la regresión lineal para el entrenamiento del modelo.
- Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
- El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
- Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.
- Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

Prepárate para el cálculo de ganancias:
- Almacena todos los valores necesarios para los cálculos en variables separadas.
- Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.
- Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.
- Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:
- Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').
- Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.
- Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.
Calcula riesgos y ganancias para cada región:
- Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.
- Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.
- Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

## 1. Objetivo
Encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

## 2. Cargar librerias

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## 3. Leer los archivos

In [3]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

## 4. Analizar los datos

In [4]:
print('geo data 0')
print(df0.head(), '\n')
print(df0.info(), '\n')

print('geo data 1')
print(df1.info(), '\n')

print('geo data 2')
print(df2.info(), '\n')

geo data 0
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

geo data 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1 

Se pudo observar que los tres dataframes constan de las columnas id, f0, f1, f2 y product. No se presentan datos ausentes y los tipos de datos son los correctos en todas las columnas.

## 5. Entrenar el modelo

In [5]:
def entrenar_modelo(df):
    #seleccionar los features y target
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']

    #dividir los datos en un conjunto de entrenamiento y uno de validación
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

    #entrenar el modelo
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    rmse = np.sqrt(mean_squared_error(target_valid, predictions))
    mean_vol = predictions.mean()
    
    return model, predictions, target_valid, rmse, mean_vol



La función entrenar_modelo tiene como argumento un dataframe para poder entrenar y analizar cada zona geográfica. Esta función tiene como datos de salida el modelo, las predicciones, el target_valid, el error cuadratico medio y el volumen medio de reservas predicho.

## 6. Analizar los resultados

In [6]:
model_0, pred_0, valid_0, rmse_0, mean_vol_0 = entrenar_modelo(df0)
model_1, pred_1, valid_1, rmse_1, mean_vol_1 = entrenar_modelo(df1)
model_2, pred_2, valid_2, rmse_2, mean_vol_2 = entrenar_modelo(df2)

print('RMSE zona 0:', rmse_0)
print('Volumen medio predicho zona 0:', mean_vol_0)
print('\n')
print('RMSE zona 1:', rmse_1)
print('Volumen medio predicho zona 1:', mean_vol_1)
print('\n')
print('RMSE zona 2:', rmse_2)
print('Volumen medio predicho zona 2:', mean_vol_2)

RMSE zona 0: 37.5794217150813
Volumen medio predicho zona 0: 92.59256778438035


RMSE zona 1: 0.893099286775617
Volumen medio predicho zona 1: 68.728546895446


RMSE zona 2: 40.02970873393434
Volumen medio predicho zona 2: 94.96504596800489


Según las predicciones, es evidente que la región con mayor volumen medio de reservas corresponde a la zona 2, sin embargo también es la región con mayor diferencia entre los valores reales y los predichos. 
La zona 1 es la segunda región de mayores reservas predichas pero con un RMSE bastante elevado.

## 7. Calcular ganancias

In [7]:
precio = 4500
inversion = 100e6
pozos = 200

def calcular_ganancia(pred, valid):
    valid = valid.reset_index(drop=True)
    pred_sorted = pred.sort_values(ascending=False).index[:pozos]
    ganancia = valid[pred_sorted].sum() * precio - inversion
    return ganancia

ganancia_0 = calcular_ganancia(pd.Series(pred_0), valid_0)
ganancia_1 = calcular_ganancia(pd.Series(pred_1), valid_1)
ganancia_2 = calcular_ganancia(pd.Series(pred_2), valid_2)

print('Ganancia zona 0:', ganancia_0)
print('Ganancia zona 1:', ganancia_1)
print('Ganancia zona 2:', ganancia_2)

Ganancia zona 0: 33208260.43139851
Ganancia zona 1: 24150866.966815114
Ganancia zona 2: 27103499.635998324


Se eligieron los 200 pozos con mayores reservas para encontrar las ganancias en las distintas regiones. En este sentido, la zona más conveniente par el desarrollo de pozos petroleros corresponde a la zona 0. Sin embargo, es necesario calcular los riesgos y evidenciar la distribución de las ganancias según el riesgo aceptado del 2.5%, para esto empleamos bootstrapping.

## 8. Bootstrapping

In [9]:
def bootstrapping(pred, valid):

    ganancias = []
    for _ in range(1000):
        
        sampled_index = pd.Series(range(len(pred))).sample(n=500, replace=True)
        pred_subsample = pd.Series(pred).iloc[sampled_index].reset_index(drop=True)
        valid_subsample = pd.Series(valid).iloc[sampled_index].reset_index(drop=True)

        top_index = pred_subsample.nlargest(200).index
        ganancias.append(valid_subsample[top_index].sum() * precio - inversion)  
        
    return ganancias
        

def riesgos(ganancias):
    g = pd.Series(ganancias)
    media = g.mean()
    intervalo = g.quantile([0.025, 0.975])
    riesgo = (g<0).mean()*100
    return media, intervalo, riesgo

gan_0 = bootstrapping(pred_0, valid_0)
gan_1 = bootstrapping(pred_1, valid_1)
gan_2 = bootstrapping(pred_2, valid_2)

media_0, intervalo_0, riesgo_0 = riesgos(gan_0)
media_1, intervalo_1, riesgo_1 = riesgos(gan_1)
media_2, intervalo_2, riesgo_2 = riesgos(gan_2)

print(f"Zona 0: Ganancia promedio:{media_0}, intervalo de confianza 95%:{intervalo_0.values}, riesgo de pérdida:{riesgo_0}%")
print('\n')
print(f"Zona 1: Ganancia promedio:{media_1}, intervalo de confianza 95%:{intervalo_1.values}, riesgo de pérdida:{riesgo_1}%")
print('\n')
print(f"Zona 2: Ganancia promedio:{media_2}, intervalo de confianza 95%:{intervalo_2.values}, riesgo de pérdida:{riesgo_2}%")

Zona 0: Ganancia promedio:3910617.3336281776, intervalo de confianza 95%:[-1392446.39885888  8932963.95923113], riesgo de pérdida:6.2%


Zona 1: Ganancia promedio:4532940.071664492, intervalo de confianza 95%:[ 838027.4032226  8345267.36514308], riesgo de pérdida:0.8999999999999999%


Zona 2: Ganancia promedio:3818065.672154088, intervalo de confianza 95%:[-1670081.11658792  9051925.27980405], riesgo de pérdida:9.2%


## 9. Conclusiones

- La región más rentable es la zona 1, con una ganancia de $us. 4.4 millones, un intérvalo de confianza completamente positivo y el menor riesgo de pérdida (1.6%). Esta elección no coincide con la respuesta previa de la zona 0.
- La peor zona en términos de rentabilidad es la zona 0, sin embargo, la zona 2 tiene mayor riesgo de pérdida.